This notebook is intended to walk through the process of interacting with WDS and CBAS to perform a submission.  This will have functions to upload and read from data tables in WDS.  It will also have functions to list and submit "run sets" or submissions to CBAS

To use this, you will need a tsv that you want to upload to your WDS data tables.  That tsv should be uploaded to the notebook environment.

## Setup

In [1]:
import json
import os
import requests
from pprint import pprint

In [2]:
# env API urls
env = "prod"
WSM_URL = f'https://workspace.dsde-{env}.broadinstitute.org'
TSPS_URL = f'https://tsps.dsde-{env}.broadinstitute.org'
ORCH_URL = f'https://firecloud-orchestration.dsde-{env}.broadinstitute.org/'
LEONARDO_URL= f'https://leonardo.dsde-{env}.broadinstitute.org/'

# retrieve info for this workspace
ws_id = os.getenv('WORKSPACE_ID')

print(f'destination workspace ID:                  {ws_id}')

workspace ID:                  7a6ab368-c165-48ae-8b9a-a2cf1138ef1c


In [3]:
def get_access_token(verbose=False):
    """Get access token for pet managed identity in Azure."""
    if verbose:
        !az login --identity --allow-no-subscriptions
    else:
        !az login --identity --allow-no-subscriptions --output none
    cli_token = !az account get-access-token | jq .accessToken

    return cli_token[0].replace('"', '')

def get_headers(verb='GET', verbose=False):
    headers = {
        'Authorization': 'Bearer ' + get_access_token(verbose),
        'accept': '*/*'
    }
    
    if verb == 'POST':
        headers['Content-Type'] = 'application/json'

    return headers

In [4]:
# get headers once - refresh this as needed
HEADERS_GET = get_headers('GET')
HEADERS_POST = get_headers('POST')

# workspace functions
def get_workspace_id(ws_project, ws_name, orch_url=ORCH_URL, verbose=False):
    uri = f'{orch_url}/api/workspaces/{ws_project}/{ws_name}'
    response = requests.get(uri, headers=HEADERS_GET)
    return response.json()['workspace']['workspaceId']

# leonardo functions
def get_apps_for_workspace_id(workspace_id,
                              leonardo_url=LEONARDO_URL,
                              verbose=False):
    uri = f'{leonardo_url}/api/apps/v2/{workspace_id}?includeDeleted=false'
    response = requests.get(uri, headers=HEADERS_GET)
    status_code = response.status_code
    
    return response.json()

def get_app_for_workspace_id(workspace_id,
                             app_name,
                             leonardo_url=LEONARDO_URL,
                             verbose=False):
    uri = f'{leonardo_url}/api/apps/v2/{workspace_id}?includeDeleted=false'
    response = requests.get(uri, headers=HEADERS_GET)
    status_code = response.status_code
    
    return response.json()

# wds functions
def get_types(instance_id,
              wds_url,
              api_version="0.2",
              verbose=False):
    uri = f'{wds_url}/{instance_id}/types/v{api_version}'
    response = requests.get(uri, headers=HEADERS_GET)
    status_code = response.status_code
    
    return response.json()
              
def query_records_for_type(instance_id,
                           type_name,
                           wds_url,
                           api_version="0.2",
                           offset=0,
                           limit=100,
                           sort="asc",
                           verbose=False):
    uri = f'{wds_url}/{instance_id}/search/v{api_version}/{type_name}'
    
    
    body = json.dumps({
        'offset': offset,
        'limit': limit,
        'sort': sort
    })
    
    response = requests.post(uri, headers=HEADERS_POST, data=body)
    status_code = response.status_code
    
    return response.json()

def upload_records_for_type(instance_id,
                            type_name,
                            wds_url,
                            tsv_file_path,
                            api_version="0.2",
                            verbose=False):
    uri = f'{wds_url}/{instance_id}/tsv/v{api_version}/{type_name}'
    test_file = open(tsv_file_path, "rb")
    
    response = requests.post(uri, headers=HEADERS_GET, files = {"records": test_file})
    status_code = response.status_code
    
    return response.json()

# cbas functions
def create_method_with_github_link(cbas_url,
                                   method_name,
                                   github_url,
                                   method_version,
                                   method_source="GitHub",
                                   verbose=False):
    
    uri = f'{cbas_url}/api/batch/v1/methods'
    post_body = json.dumps({
        'method_name': method_name,
        'method_source': method_source,
        'method_version': method_version,
        'method_url': github_url
    })
    
    response = requests.post(uri, headers=HEADERS_POST, data=post_body)
    status_code = response.status_code
    
    return response.json()

def get_methods(cbas_url,
                show_versions=True,
                verbose=False):
    
    uri = f'{cbas_url}/api/batch/v1/methods?show_versions={show_versions}'
    response = requests.get(uri, headers=HEADERS_GET)
    status_code = response.status_code
    
    return response.json()

def get_method_info(cbas_url,
                    method_id,
                    verbose=False):
    
    uri = f'{cbas_url}/api/batch/v1/methods?method_id={method_id}'
    response = requests.get(uri, headers=HEADERS_GET)
    status_code = response.status_code
    
    return response.json()

def get_run_sets_for_method(cbas_url,
                            method_id,
                            page_size=1,
                            verbose=False):
    
    uri = f'{cbas_url}/api/batch/v1/run_sets?method_id={method_id}&page_size=1'
    response = requests.get(uri, headers=HEADERS_GET)
    status_code = response.status_code
    
    return response.json()

def submit_run_set(cbas_url,
                   post_body,
                   verbose=False):
    
    uri = f'{cbas_url}/api/batch/v1/run_sets'
    
    response = requests.post(uri, headers=HEADERS_POST, data=post_body)
    status_code = response.status_code
    
    return response.json()

## SETUP WDS and CBAS URIs (need to run before running downstream functions)

In [5]:
# first we need to get the URIs to the wds and cbas apps spun up in our workspace.  this will allow us to make api calls to these apps

cbas_uri = ''
wds_uri = ''
get_apps_response = get_apps_for_workspace_id(ws_id)
for app in get_apps_response:
    if app['appType'] == 'CROMWELL':
       cbas_uri = app['proxyUrls']['cbas']
    if app['appType'] == 'WDS':
       wds_uri = app['proxyUrls']['wds']

# in order to find the swagger page for cbas, add /swagger-ui.html to the cbas_uri
print(f'cbas_uri: {cbas_uri}')


# the wds swagger page can be found at https://petstore.swagger.io/?url=https://raw.githubusercontent.com/DataBiosphere/terra-workspace-data-service/main/service/src/main/resources/static/swagger/openapi-docs.yaml
# there currently is no love swagger page you can hit.
print(f'wds_url: {wds_uri}')


       

cbas_uri: https://lzf42bf0dc63c251179adc6ee67ef00d66ba53808001a58d33.servicebus.windows.net/terra-app-7e72dad7-3ec0-4ce9-a767-aeee668c1b45-7a6ab368-c165-48ae-8b9a-a2cf1138ef1c/cbas
wds_url: https://lzf42bf0dc63c251179adc6ee67ef00d66ba53808001a58d33.servicebus.windows.net/wds-7a6ab368-c165-48ae-8b9a-a2cf1138ef1c-7a6ab368-c165-48ae-8b9a-a2cf1138ef1c/


In [59]:
## WDS stuff

In [60]:
# look up what tables exist in current wds
get_types_response = get_types(ws_id, wds_uri)

print(f'get types response: {get_types_response}')

# for each table in our WDS, query records in that table
for table in get_types_response:
    table_name = table['name']
    print(f'table name: {table_name}')
    print(f'records for table: {query_records_for_type(ws_id, table_name, wds_uri)}')

# given a tsv and a table name, upload to WDS - modify these to match your values
table_name = 'notebook_v2_array'
tsv_path = 'notebook_array.tsv'
print(f'upload tsv to WDS response: {upload_records_for_type(ws_id, table_name, wds_uri, tsv_path)}')

# if you uploaded a net new table you should see it now in this response
get_types_response = get_types(ws_id, wds_uri)
for table in get_types_response:
    table_name = table['name']
    print(f'table name: {table_name}')

get types response: [{'name': 'multisample_array', 'attributes': [{'name': 'aggregated_imputation_metrics', 'datatype': 'FILE'}, {'name': 'chunks_info', 'datatype': 'FILE'}, {'name': 'failed_chunks', 'datatype': 'FILE'}, {'name': 'imputed_multisample_vcf', 'datatype': 'FILE'}, {'name': 'imputed_multisample_vcf_index', 'datatype': 'FILE'}, {'name': 'multisample_array_id', 'datatype': 'STRING'}, {'name': 'multisample_array_vcf', 'datatype': 'FILE'}, {'name': 'multisample_array_vcf_index', 'datatype': 'FILE'}, {'name': 'n_failed_chunks', 'datatype': 'FILE'}, {'name': 'phase_eagle_genetic_map', 'datatype': 'FILE'}, {'name': 'ref_dict', 'datatype': 'STRING'}, {'name': 'ref_panel_path', 'datatype': 'STRING'}], 'count': 1, 'primaryKey': 'multisample_array_id'}, {'name': 'notebook_v2_array', 'attributes': [{'name': 'multisample_array_vcf', 'datatype': 'FILE'}, {'name': 'multisample_array_vcf_index', 'datatype': 'FILE'}, {'name': 'notebook_array_id', 'datatype': 'STRING'}, {'name': 'phase_eagle

In [7]:
# upload table to WDS in another workspace, modify these variables to match your destination workspace
dest_ws_project = 'dsp-azure-general'
dest_ws_name = 'js-dest-wds'


dest_ws_id = get_workspace_id(dest_ws_project, dest_ws_name)
print(f'destination workspace id: {dest_ws_id}')

dest_wds_uri = ''
get_apps_response = get_apps_for_workspace_id(dest_ws_id)
for app in get_apps_response:
    if app['appType'] == 'WDS':
       dest_wds_uri = app['proxyUrls']['wds']


# given a tsv and a table name, upload to WDS - modify these to match your values
table_name = 'notebook_v2_array'
tsv_path = 'notebook_array.tsv'
print(f'upload tsv to WDS response: {upload_records_for_type(dest_ws_id, table_name, dest_wds_uri, tsv_path)}')

destination workspace id: fc9e54b0-2123-4cab-813b-9607fbfc21c1
{'recordsModified': 1, 'message': 'Updated notebook_v2_array'}


In [61]:
## CBAS stuff

In [62]:
# get method id for an existing method (once we upload our method we will know what this is) and its latest method_version_id

# create a cbas method given a github url
method_name = 'dont_use_me_either'
github_url = 'https://github.com/broadinstitute/warp/blob/js_try_imputation_azure/pipelines/broad/arrays/imputation/Imputation.wdl'
method_version = "1.0"

create_method_with_github_link(cbas_uri, method_name, github_url, method_version)

# for the newly created method, get its method_id and method_version_id used for downstream functions
method_id = ""
method_version_id = ""
get_methods_response = get_methods(cbas_uri)
for method in get_methods_response['methods']:
    if method['name'] == method_name:
        method_id = method['method_id']
        method_version_id = method['method_versions'][0]['method_version_id']
        
print(f'method id: {method_id}')
print(f'method version id: {method_version_id}')

# get info for newly created method
get_method_info(cbas_uri, method_id)

# in order to get the workflow_input_description and workflow_output_description that is used when generating a submission you can call the run_sets endpoint with your method id
run_set_response = get_run_sets_for_method(cbas_uri, method_id)
run_set = run_set_response['run_sets'][0]
workflow_input_definition = run_set['input_definition']
workflow_output_definition = run_set['output_definition']

# these can be very large so commenting them out by default

# print(f'input definition: {workflow_input_definition}')
# print(f'output definition: {workflow_output_definition}')



# here is an example submission post body. some thigns to note are the use of the method_version_id and the patterns used
# when inputs/outputs are sourced/outputted from/to the WDS table and when they are not.
# also worth noting that all inputs/outputs are specified even if no value is passed which may not be necessary - need to follow up with cbas team.
imputation_wdl_submission_post = '{"run_set_name":"imputation-azure-v-6_multisample-array_notebook-submit","run_set_description":"submit from notebook","method_version_id":"f0b05206-dab4-4bff-9450-968c6753a7ff","workflow_input_definitions":[{"input_name":"Imputation.AggregateImputationQCMetrics.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.AggregateImputationQCMetrics.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.AggregateImputationQCMetrics.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.AggregateImputationQCMetrics.rtidyverse_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateChromosomeLength.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateChromosomeLength.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateChromosomeLength.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateChromosomeLength.ubuntu_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateMissingChromosomeLength.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateMissingChromosomeLength.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateMissingChromosomeLength.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CalculateMissingChromosomeLength.ubuntu_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.CheckChunks.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.CheckChunks.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CheckChunks.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CheckChunks.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CountSamples.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.CountSamples.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CountSamples.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CountSamples.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CountVariantsInChunks.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CountVariantsInChunks.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.CountVariantsInChunks.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.CountVariantsInChunks.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIDs.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIDs.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIDs.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIDs.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIdsVcfToImpute.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIdsVcfToImpute.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIdsVcfToImpute.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ExtractIdsVcfToImpute.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.FindSitesUniqueToFileTwoOnly.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.FindSitesUniqueToFileTwoOnly.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.FindSitesUniqueToFileTwoOnly.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.FindSitesUniqueToFileTwoOnly.ubuntu_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.GatherVcfs.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GatherVcfs.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GatherVcfs.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.GatherVcfs.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GenerateChunk.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GenerateChunk.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GenerateChunk.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.GenerateChunk.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GetMissingContigList.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GetMissingContigList.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GetMissingContigList.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.GetMissingContigList.ubuntu_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.InterleaveVariants.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.InterleaveVariants.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.InterleaveVariants.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.InterleaveVariants.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeImputationQCMetrics.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeImputationQCMetrics.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeImputationQCMetrics.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeImputationQCMetrics.rtidyverse_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeSingleSampleVcfs.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeSingleSampleVcfs.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.MergeSingleSampleVcfs.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.Minimac4.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.Minimac4.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.Minimac4.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.Minimac4.minimac4_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.OptionalQCSites.bcftools_vcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.OptionalQCSites.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.OptionalQCSites.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.OptionalQCSites.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.PhaseVariantsEagle.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.PhaseVariantsEagle.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.PhaseVariantsEagle.eagle_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.PhaseVariantsEagle.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotations.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotations.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotations.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotations.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotationsMissingContigs.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotationsMissingContigs.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotationsMissingContigs.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveAnnotationsMissingContigs.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveSymbolicAlleles.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveSymbolicAlleles.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveSymbolicAlleles.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.RemoveSymbolicAlleles.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.ReplaceHeader.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SelectVariantsByIds.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SelectVariantsByIds.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SelectVariantsByIds.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SelectVariantsByIds.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SeparateMultiallelics.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SeparateMultiallelics.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SeparateMultiallelics.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SeparateMultiallelics.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDs.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDs.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDs.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDs.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDsMissingContigs.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDsMissingContigs.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDsMissingContigs.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIDsMissingContigs.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIdsVcfToImpute.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIdsVcfToImpute.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIdsVcfToImpute.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SetIdsVcfToImpute.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SplitMultiSampleVcf.bcftools_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SplitMultiSampleVcf.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SplitMultiSampleVcf.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SplitMultiSampleVcf.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.StoreChunksInfo.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.StoreChunksInfo.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.StoreChunksInfo.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.StoreChunksInfo.rtidyverse_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegion.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegion.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegion.exclude_filtered","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Boolean"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegion.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegion.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegionMissingContig.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegionMissingContig.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegionMissingContig.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.SubsetVcfToRegionMissingContig.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.UpdateHeader.cpu","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.UpdateHeader.disk_size_gb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.UpdateHeader.gatk_docker","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.UpdateHeader.memory_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.bcf_index_suffix","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.bcf_suffix","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.chunkLength","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.chunkOverlaps","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.chunks_fail_threshold","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.contigs","input_type":{"type":"array","non_empty":false,"array_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"literal","parameter_value":["20"]}},{"input_name":"Imputation.frac_above_maf_5_percent_well_imputed_threshold","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Float"}},"source":{"type":"none"}},{"input_name":"Imputation.genetic_maps_eagle","input_type":{"type":"primitive","primitive_type":"File"},"source":{"type":"record_lookup","record_attribute":"phase_eagle_genetic_map"}},{"input_name":"Imputation.m3vcf_suffix","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.merge_ssvcf_mem_mb","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Int"}},"source":{"type":"none"}},{"input_name":"Imputation.multi_sample_vcf","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"File"}},"source":{"type":"record_lookup","record_attribute":"multisample_array_vcf"}},{"input_name":"Imputation.multi_sample_vcf_index","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"File"}},"source":{"type":"record_lookup","record_attribute":"multisample_array_vcf_index"}},{"input_name":"Imputation.optional_qc_hwe","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Float"}},"source":{"type":"literal","parameter_value":0.000001}},{"input_name":"Imputation.optional_qc_max_missing","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Float"}},"source":{"type":"literal","parameter_value":0.05}},{"input_name":"Imputation.output_callset_name","input_type":{"type":"primitive","primitive_type":"String"},"source":{"type":"record_lookup","record_attribute":"multisample_array_id"}},{"input_name":"Imputation.perform_extra_qc_steps","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Boolean"}},"source":{"type":"none"}},{"input_name":"Imputation.ref_dict","input_type":{"type":"primitive","primitive_type":"File"},"source":{"type":"record_lookup","record_attribute":"ref_dict"}},{"input_name":"Imputation.reference_panel_path","input_type":{"type":"primitive","primitive_type":"String"},"source":{"type":"record_lookup","record_attribute":"ref_panel_path"}},{"input_name":"Imputation.single_sample_vcf_indices","input_type":{"type":"optional","optional_type":{"type":"array","non_empty":false,"array_type":{"type":"primitive","primitive_type":"File"}}},"source":{"type":"none"}},{"input_name":"Imputation.single_sample_vcfs","input_type":{"type":"optional","optional_type":{"type":"array","non_empty":false,"array_type":{"type":"primitive","primitive_type":"File"}}},"source":{"type":"none"}},{"input_name":"Imputation.split_output_to_single_sample","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"Boolean"}},"source":{"type":"none"}},{"input_name":"Imputation.vcf_index_suffix","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}},{"input_name":"Imputation.vcf_suffix","input_type":{"type":"optional","optional_type":{"type":"primitive","primitive_type":"String"}},"source":{"type":"none"}}],"workflow_output_definitions":[{"output_name":"Imputation.aggregated_imputation_metrics","output_type":{"type":"primitive","primitive_type":"File"},"destination":{"type":"record_update","record_attribute":"aggregated_imputation_metrics"}},{"output_name":"Imputation.chunks_info","output_type":{"type":"primitive","primitive_type":"File"},"destination":{"type":"record_update","record_attribute":"chunks_info"}},{"output_name":"Imputation.failed_chunks","output_type":{"type":"primitive","primitive_type":"File"},"destination":{"type":"record_update","record_attribute":"failed_chunks"}},{"output_name":"Imputation.imputed_multisample_vcf","output_type":{"type":"primitive","primitive_type":"File"},"destination":{"type":"record_update","record_attribute":"imputed_multisample_vcf"}},{"output_name":"Imputation.imputed_multisample_vcf_index","output_type":{"type":"primitive","primitive_type":"File"},"destination":{"type":"record_update","record_attribute":"imputed_multisample_vcf_index"}},{"output_name":"Imputation.imputed_single_sample_vcf_indices","output_type":{"type":"optional","optional_type":{"type":"array","non_empty":false,"array_type":{"type":"primitive","primitive_type":"File"}}},"destination":{"type":"record_update","record_attribute":"imputed_single_sample_vcf_indices"}},{"output_name":"Imputation.imputed_single_sample_vcfs","output_type":{"type":"optional","optional_type":{"type":"array","non_empty":false,"array_type":{"type":"primitive","primitive_type":"File"}}},"destination":{"type":"record_update","record_attribute":"imputed_single_sample_vcfs"}},{"output_name":"Imputation.n_failed_chunks","output_type":{"type":"primitive","primitive_type":"File"},"destination":{"type":"record_update","record_attribute":"n_failed_chunks"}}],"wds_records":{"record_type":"multisample_array","record_ids":["palantir_42_samples"]}}'

# this will submit a submission in a specific workspace. as a POC didnt need to make this super generic.
#print(f'submit response: {submit_run_set(cbas_uri, imputation_wdl_submission_post)}')

method id: 7be4f1ca-563e-42c1-8305-650915b3e193
method version id: f6749b2d-4ac0-4ea8-8208-f9ab8f123967
